## Sliding window intensity assignment method, all (working) versions:
- i_sw 0.1 works, but leaves white edges, where the window doesn't reach

In [3]:
#run this for the codes to work :)

%pylab
%matplotlib inline
import numpy as np
from scipy import stats
import scipy.ndimage
import matplotlib.pyplot as plt
from images_segmentation import otsu as ot

def crop(image, xmin, ymin, xmax, ymax):
    cropped=np.empty([xmax-xmin, ymax-ymin], dtype=float)
    for a,b in np.ndindex(xmax-xmin, ymax-ymin):
        cropped[a,b]=image[a+xmin,b+ymin]
    return cropped

def otsu_i(image, selectivity):
    img=ot.otsu_thresholding(image, 256)
    it = np.zeros([img.shape[0],img.shape[1],2])
    for x, y in np.ndindex(img.shape[0],img.shape[1]):
            if img[x,y] == 0:
                it[x,y,0]+=1
            else: 
                it[x,y,0]+=0
            it[x,y,1]+=selectivity
    return it

def edge(image, framesize):
    img = np.empty([image.shape[0]+framesize, image.shape[1]+framesize], dtype=int)
    for i,j in np.ndindex(image.shape[0],image.shape[1]):
        img[i,j]=image[i,j]
    figure()
    imshow(img,'gray')
    return img


Using matplotlib backend: <object object at 0x00000257AFED2600>
%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


## i_sw 0.1

In [5]:
def i_sw(image,stepsize,framesize, sensitivity):
    img=copy(image)
    for i,j in np.ndindex(image.shape[0], image.shape[1]):
        img[i,j]=image[i,j]
    
    #create a 3 dimensional null array, the size of the image * 2, we can now save 2 separate values for each pixel
    array=np.empty([img.shape[0],img.shape[1],2], dtype=float)

    #set the first coordinates for iteration over the image
    x=0
    y=0

        #iterate crop function with a given size over the picture, while the "window" fits inside the image, 
        #for each window crop the image to window coordinates and perform otsu_i 
    
    while x+framesize<=img.shape[0]:
        while y+framesize<=img.shape[1]:
            window=crop(img,x,y,x+framesize, y+framesize)
            it = otsu_i(window, sensitivity)

            # for each pixel that fits into the current window, add the values from otsu_i 
            #generated array to according pixels in the big array
            for a, b in np.ndindex(window.shape[0], window.shape[1]):
                c=a+x
                d=b+y
                array[c,d,0]+=it[a,b,0]
                array[c,d,1]+=it[a,b,1]
            y+=stepsize
        y=0
        x+=stepsize

    # because array is 3 dimensional and picture is 2 dimensional the final intensities will
    # be determined by 'array', but saved into a final 'img' array (rewriting the copy of image)
    for i, j in np.ndindex(img.shape[0], img.shape[1]):
        if array[i,j,0]>array[i,j,1]:
            img[i,j]=0
        else:
            img[i,j]=255
    img=crop(img,0,0,image.shape[0],image.shape[1])
    figure()
    imshow(img,'gray')
    return img